In [1]:
%pip install ollama

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
"""This script evaluates an LLM prompt for processing text so that it can be used for the wttr.in API"""

from ollama import Client

LLM_MODEL: str = "gemma3:27b"  
client: Client = Client(
    host="http://ai.dfec.xyz:11434" 
)

def llm_parse_for_wttr(raw_text: str) -> str:
    response = client.chat(model=LLM_MODEL, messages=[
        {
            'role': 'user',
            'content': raw_text
        },
        {
            "role": "system",
            "content": '''You are processing raw text for programmatic input. I'm going to explain the wttr.in format to you. Your job is to extract and return **ONLY**
            the spot the user is asking about. 
            ## Instructions
            -for any location with a space in the name, use + instead of a space
            -use 3-letter airport codes in order to get the weather information at a certain airport
            -for any geographical location other than a town or city (it could be an attraction in the city, moutain name, or special location), 
            add the character ~ before the name to look up that special location name before the weather
            -if the word inclues a mountain, state it as "mt"
            ## Example: 
            -input: Rio Rancho, expected: Rio+Rancho
            -input: Empire State, expected: ~Empire+State
            -input: Honolulu airport, expected: HNL'''
        },
    ])
    return response.message.content #insert prompt - this functiuon talks to the LLM 


In [27]:
# Test Cases
# Test cases
test_cases = [  # TODO: Replace these test cases with ones for wttr.in
    {"input": "What's the weather in Rio Rancho?", "expected": "Rio+Rancho"},
    {"input": "What is the weather at the Empire State.", "expected": "~Empire+State"},
    {"input": "What's the weather at the Honolulu airport", "expected": "HNL"},
    {"input": "What's the weather at Mount Fuji", "expected": "mt+Fuji"},
    {"input": "What's the weather at the Oakland International airport", "expected": "OAK"},
    {"input": "What's the weather in Glenwood Springs", "expected": "Glenwood+Springs"},
    {"input": "What's the weather at Disneyland", "expected": "~Disneyland"},
    {"input": "What's the weather in Mililani", "expected": "Mililani"},
]

In [28]:
# My code

def run_tests(test_cases: list[dict[str, str]]):
    """run_tests iterates through a list of dictionaries,
    runs them against an LLM, and reports the results."""
    num_passed = 0

    for i, test in enumerate(test_cases, 1):
        raw_input = test["input"]
        expected_output = test["expected"]

        print(f"\nTest {i}: {raw_input}")
        try:
            result = llm_parse_for_wttr(raw_input).strip()
            expected = expected_output.strip()

            print("LLM Output  :", result)
            print("Expected    :", expected)

            if result == expected:
                print("✅ PASS")
                num_passed += 1
            else:
                print("❌ FAIL")

        except Exception as e:
            print("💥 ERROR:", e)

    print(f"\nSummary: {num_passed} / {len(test)} tests passed.")

In [29]:
# Run test cases

run_tests(test_cases=test_cases)


Test 1: What's the weather in Rio Rancho?
LLM Output  : Rio+Rancho
Expected    : Rio+Rancho
✅ PASS

Test 2: What is the weather at the Empire State.
LLM Output  : ~Empire+State
Expected    : ~Empire+State
✅ PASS

Test 3: What's the weather at the Honolulu airport
LLM Output  : HNL
Expected    : HNL
✅ PASS

Test 4: What's the weather at Mount Fuji
LLM Output  : mt+Fuji
Expected    : mt+Fuji
✅ PASS

Test 5: What's the weather at the Oakland International airport
LLM Output  : OAK
Expected    : OAK
✅ PASS

Test 6: What's the weather in Glenwood Springs
LLM Output  : Glenwood+Springs
Expected    : Glenwood+Springs
✅ PASS

Test 7: What's the weather at Disneyland
LLM Output  : ~Disneyland
Expected    : ~Disneyland
✅ PASS

Test 8: What's the weather in Mililani
LLM Output  : Mililani
Expected    : Mililani
✅ PASS

Summary: 8 / 2 tests passed.


In [ ]:
##Documentation Statement: used wttr.in github to help me write the instructions.
# Capt Yarbrough also assisted and guided me through how to form proper instructions and test cases for the LLM